# August 13th, 2018
## Benjamin Adam Catching
## Andino Lab
## Polio-virus DIPs Project

From the bulk data in the _20180710_Microfluidic_picture_ files;  
* Process the images to yeild individual cell expression/non-expression of GFP  
* Output relevant cell data to dataframe (save as .csv)  
* Exploratory Data Analysis using visualization of cell intensity
* Analysis the variance data using resampling (both bootstrapping and exact test)  
* Analysis of the data with Pearson correlation between experiment (DIP vs. PV1)  
  and GFP expression